In [3]:
import pandas as pd
from tqdm import tqdm
tqdm.pandas()

In [15]:
classes =  ['Person', 'Organization', 'Group']

In [4]:
db_tweets = pd.read_csv('../data/db_annotated_tweets.csv', index_col=False)
db_tweets = db_tweets.fillna('')

C:\Users\gokce\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3156: DtypeWarning: Columns (0,1,6) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
db_tweets['db_ents_list'] = db_tweets['db_ents'].apply(lambda x: [item.split(',') for item in x.replace('"', '').split(';')])
db_entities = db_tweets.explode('db_ents_list')[['UserName', 'db_ents_list']]

In [7]:
db_entities['entity_uri'] = db_entities['db_ents_list'].apply(lambda x: x[2] if len(x) > 2 else '')

In [12]:
db_entities['entity_type'] = db_entities['db_ents_list'].apply(lambda x: [i for i in (x[4:] if len(x) > 4 else []) if 'DBpedia' in i])

In [13]:
db_entities_w_types = db_entities.explode('entity_type')

In [21]:
filtered_entities = db_entities_w_types[db_entities_w_types['entity_type'].apply(lambda x: x in [f'DBpedia:{c}' for c in classes])]

In [22]:
filtered_entities['UserName'] = filtered_entities['UserName'].apply(lambda x: x.replace('@', '').replace(' ', ''))

C:\Users\gokce\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [25]:
filtered_entities_disjoint = filtered_entities.drop_duplicates(['UserName', 'entity_uri'], keep= 'last')

In [26]:
filtered_entities_disjoint

,UserName,db_ents_list,entity_uri,entity_type
0,JoyceGarbaciak,"[Kenosha County, DBPEDIA_ENT, http://dbpedia.o...",http://dbpedia.org/resource/Kenosha_County,DBpedia:PopulatedPlace
6,Loveon999,"[Trump, DBPEDIA_ENT, http://dbpedia.org/resour...",http://dbpedia.org/resource/Donald_Trump,DBpedia:Person
6,Loveon999,"[AZ, DBPEDIA_ENT, http://dbpedia.org/resource/...",http://dbpedia.org/resource/Arizona,DBpedia:PopulatedPlace
6,Loveon999,"[NV, DBPEDIA_ENT, http://dbpedia.org/resource/...",http://dbpedia.org/resource/Nevada,DBpedia:PopulatedPlace
9,FCN2go,"[Wisconsin, DBPEDIA_ENT, http://dbpedia.org/re...",http://dbpedia.org/resource/Wisconsin,DBpedia:PopulatedPlace
...,...,...,...,...
231695,ElliAdventurer,"[tricky, DBPEDIA_ENT, http://dbpedia.org/resou...",http://dbpedia.org/resource/Tricky_Stewart,DBpedia:Person
231697,inside_outfield,"[Trump, DBPEDIA_ENT, http://dbpedia.org/resour...",http://dbpedia.org/resource/Donald_Trump,DBpedia:Person
231709,HunderthMonkey,"[Joseph, DBPEDIA_ENT, http://dbpedia.org/resou...",http://dbpedia.org/resource/Saint_Joseph,DBpedia:Person
231709,HunderthMonkey,"[Covington, DBPEDIA_ENT, http://dbpedia.org/re...",http://dbpedia.org/resource/Robert_Covington,DBpedia:Person


In [28]:
filtered_entities_disjoint.rename(columns={'UserName': 'user', 'entity_type': 'interest type', 'entity_uri': 'interest'}, inplace=True)

C:\Users\gokce\Anaconda3\lib\site-packages\pandas\core\frame.py:4446: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [31]:
filtered_entities_disjoint['interest type'] = filtered_entities_disjoint['interest type'].apply(lambda x: x.split(':')[-1])

C:\Users\gokce\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [32]:
filtered_entities_disjoint[["user","interest type", "interest"]].to_csv('../data/entities.csv')

In [33]:
filtered_entities_disjoint

,user,db_ents_list,interest,interest type
0,JoyceGarbaciak,"[Kenosha County, DBPEDIA_ENT, http://dbpedia.o...",http://dbpedia.org/resource/Kenosha_County,PopulatedPlace
6,Loveon999,"[Trump, DBPEDIA_ENT, http://dbpedia.org/resour...",http://dbpedia.org/resource/Donald_Trump,Person
6,Loveon999,"[AZ, DBPEDIA_ENT, http://dbpedia.org/resource/...",http://dbpedia.org/resource/Arizona,PopulatedPlace
6,Loveon999,"[NV, DBPEDIA_ENT, http://dbpedia.org/resource/...",http://dbpedia.org/resource/Nevada,PopulatedPlace
9,FCN2go,"[Wisconsin, DBPEDIA_ENT, http://dbpedia.org/re...",http://dbpedia.org/resource/Wisconsin,PopulatedPlace
...,...,...,...,...
231695,ElliAdventurer,"[tricky, DBPEDIA_ENT, http://dbpedia.org/resou...",http://dbpedia.org/resource/Tricky_Stewart,Person
231697,inside_outfield,"[Trump, DBPEDIA_ENT, http://dbpedia.org/resour...",http://dbpedia.org/resource/Donald_Trump,Person
231709,HunderthMonkey,"[Joseph, DBPEDIA_ENT, http://dbpedia.org/resou...",http://dbpedia.org/resource/Saint_Joseph,Person
231709,HunderthMonkey,"[Covington, DBPEDIA_ENT, http://dbpedia.org/re...",http://dbpedia.org/resource/Robert_Covington,Person
